In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import os

from IPython.display import HTML
from IPython.display import display_html

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from utils.utils import *

import networkx as nx

In [2]:
FN = "../data/cosmos09/raw_views_ophashes_2021-12-15.ss.csv"
#DEDUPFN = FN.replace(".ss.csv", "-dedup.ss.csv")
#INP_FIELD = "NormInputHashCode"
#FILTER_FIELD = "FilterExpr"

In [3]:
df = pd.read_csv(FN, sep="\t")

In [4]:
print(df.keys())

Index(['JobGUID', 'HashTag', 'HashTagInput', 'HashTagRoot', 'HashTagParent',
       'OperatorName'],
      dtype='object')


In [ ]:
df.groupby("OperatorName").describe(percentiles=[0.9,0.99])["HashTag"].reset_index().sort_values(by="count",
                                                                                                ascending=False)

In [23]:
print(len(df))
print(len(set(df["JobGUID"])))
print(len(set(df["HashTagRoot"])))
print(len(set(df["OperatorName"])))

61169443
1727268
2500708
29


In [24]:
print(set(df["OperatorName"]))

{'PhyOp_ComputeScalar', 'PhyOp_Filter', 'PhyOp_RestrRemap', 'PhyOp_Sort', 'PhyOp_Spool', 'PhyOp_Process', 'PhyOp_ConstTableScan', 'PhyOp_HashGbAgg', 'PhyOp_Top', 'PhyOp_Extract', 'PhyOp_NOP', 'PhyOp_UnionAll', 'PhyOp_HashJoin', 'PhyOp_VirtualDataset', 'PhyOp_GbApply', 'PhyOp_Range', 'PhyOp_MergeJoin', 'PhyOp_Exchange', 'PhyOp_StreamGbAgg', 'PhyOp_LoopsJoin', 'PhyOp_Output', 'PhyOp_ComputeSequence', 'PhyOp_Reduce', 'PhyOp_Combine', 'PhyOp_Write', 'PhyOp_Update', 'PhyOp_Window', 'PhyOp_IdxLookup', 'PhyOp_Sampler'}


In [25]:
jguids = df["JobGUID"].sample(500).values

In [26]:
df = df[df["JobGUID"].isin(jguids)]

In [28]:
print(df.keys())

Index(['JobGUID', 'HashTag', 'HashTagInput', 'HashTagRoot', 'HashTagParent',
       'OperatorName'],
      dtype='object')


In [ ]:
def make_graph(df):
    G = nx.Graph()
    #roots = df["HashTagRoot"]
    for _,row in df.iterrows():
        node = row["HashTag"]
        G.add_node(node)
        if row["HashTag"] == row["HashTagRoot"]:
            G.nodes()[node]["root"] = True
            assert False
        else:
            G.nodes()[node]["root"] = False
        G.nodes()[node]["op"] = row["OperatorName"]
    
    for _,row in df.iterrows():
        if row["HashTagParent"] not in G.nodes():
            G.add_node(row["HashTagParent"])
            G.nodes()[row["HashTagParent"]]["root"] = False
            G.nodes()[row["HashTagParent"]]["op"] = "top"
            assert row["HashTagParent"] != row["HashTagRoot"]
        
        assert row["HashTag"] in G.nodes()
        assert row["HashTagParent"] in G.nodes()
        G.add_edge(row["HashTagParent"], row["HashTag"])

    toremove = []
    for node in G.nodes():
        assert "op" in G.nodes()[node]
#         if G.in_degree(node) == 0 and G.out_degree(node) == 0:
#             toremove.append(node)
        if G.degree(node) == 0:
            toremove.append(node)
            
    for node in toremove:
        G.remove_node(node)
    
    return G

In [ ]:
nodelabels = {}
nodelabels["join"] = "red"
nodelabels["root"] = "blue"
nodelabels["gbapply"] = "grey"

In [ ]:
def draw_plan(G):
    plt.figure(figsize=(18,6)) 
    pos = nx.nx_pydot.pydot_layout(G, prog="dot")
    colors = {}
    colors = []
    for node in G.nodes():
        if "join" in G.nodes()[node]["op"].lower():
            #colors[node] = nodelabels["join"]
            colors.append(nodelabels["join"])
        elif G.nodes()[node]["root"]:
            #colors[node] = nodelabels["root"]
            colors.append(nodelabels["root"])
        elif "gbapply" in G.nodes()[node]["op"].lower():
            #colors[node] = nodelabels["gbapply"]
            colors.append(nodelabels["gbapply"])
        else:
            #colors[node] = "black"
            colors.append("black")
                
    # make empty plot with correct color and label for each group
    nx.draw(G, pos=pos, 
            node_size=1000,
            node_color=colors,
            font_size=12)
    for k,v in nodelabels.items():
        plt.scatter([],[], c=v, label=k)
    plt.legend()
    plt.show()

In [114]:
for jobguid in set(df["JobGUID"]):
    tmp = df[df["JobGUID"] == jobguid]
    G = make_graph(tmp)
    #draw_plan(G)